In [1]:
import logging

import probtorch
import torch

import combinators
import smc
import hmm
import utils

logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [2]:
generative = smc.ParticleTrace(1)

In [3]:
hmm_params = utils.vardict({
    'mu': {
        'loc': torch.arange(5, dtype=torch.float).unsqueeze(0) * 2,
        'scale': torch.ones(1, 5) * 0.25,
    },
    'sigma': {
        'loc': torch.ones(1, 5),
        'scale': torch.ones(1, 5) * 0.25,
    }
})
for k in range(6):
    hmm_params['Pi_%d' % k] = {'concentration': torch.ones(1, 5)}

In [4]:
init_hmm = combinators.Model(f=hmm.init_hmm, theta=hmm_params)
init_hmm.condition(generative)

In [5]:
z0, mu, sigma, pi = init_hmm(T=50)

In [6]:
hmm_step = combinators.Model(f=hmm.hmm_step)
hmm_run = combinators.Model.partial(combinators.sequence, hmm_step)

In [7]:
hmm_run.condition(generative)

In [8]:
z_last, mu, sigma, pi = hmm_run(50, z0, mu, sigma, pi)

In [9]:
generative = generative.squeeze()

In [10]:
num_particles = 100

In [11]:
smc_hmm_params = utils.vardict({
    'mu': {
        'loc': torch.arange(5, dtype=torch.float) * 2,
        'scale': torch.ones(5),
    },
    'sigma': {
        'loc': torch.ones(5),
        'scale': torch.ones(5),
    }
})
for k in range(6):
    smc_hmm_params['Pi_%d' % k] = {'concentration': torch.ones(5)}

In [12]:
smc_hmm = smc.smc(hmm.hmm_step, hmm.hmm_retrace, reparameterized=False)

In [13]:
samples, elbos, inference = smc.particle_mh(num_particles, hmm.init_hmm, smc_hmm, 1000, 50, smc_hmm_params, generative, use_cuda=False)

In [14]:
for t in range(50):
    t += 1
    key = 'Z_%d' % t
    accuracy = (inference[key].value == generative[key].value).sum().to(dtype=torch.float) / inference.num_particles
    print('SMC accuracy at time %d: %f' % (t, accuracy))

SMC accuracy at time 1: 1.000000
SMC accuracy at time 2: 0.650000
SMC accuracy at time 3: 1.000000
SMC accuracy at time 4: 0.000000
SMC accuracy at time 5: 0.000000
SMC accuracy at time 6: 1.000000
SMC accuracy at time 7: 0.350000
SMC accuracy at time 8: 0.000000
SMC accuracy at time 9: 0.000000
SMC accuracy at time 10: 1.000000
SMC accuracy at time 11: 0.000000
SMC accuracy at time 12: 0.350000
SMC accuracy at time 13: 0.000000
SMC accuracy at time 14: 0.000000
SMC accuracy at time 15: 0.000000
SMC accuracy at time 16: 0.000000
SMC accuracy at time 17: 0.020000
SMC accuracy at time 18: 0.000000
SMC accuracy at time 19: 0.000000
SMC accuracy at time 20: 0.000000
SMC accuracy at time 21: 0.000000
SMC accuracy at time 22: 0.350000
SMC accuracy at time 23: 0.000000
SMC accuracy at time 24: 0.420000
SMC accuracy at time 25: 0.400000
SMC accuracy at time 26: 0.000000
SMC accuracy at time 27: 0.230000
SMC accuracy at time 28: 1.000000
SMC accuracy at time 29: 0.250000
SMC accuracy at time 30

In [15]:
elbos

tensor([-133.4281, -127.9498, -125.3607, -125.2745, -125.2745, -125.6963,
        -123.1532, -123.1532, -123.1532, -123.1532, -123.1532, -123.1532,
        -119.1347, -118.8918, -117.2164, -117.2164, -117.2164, -117.2164,
        -117.2164, -117.2164, -117.2164, -117.2164, -117.2164, -117.2164,
        -117.2164, -117.2164, -117.2164, -117.2164, -117.2164, -117.2164,
        -117.2164, -117.2164, -117.2164, -117.2164, -117.2164, -117.2164,
        -117.2164, -117.2164, -117.2164, -117.2164, -117.2164, -117.2164,
        -117.2164, -117.2164, -117.2164, -117.2164, -117.2164, -117.2164,
        -117.2164, -117.2164, -117.2164, -117.2164, -117.2164, -117.2164,
        -117.2164, -117.2164, -117.2164, -117.2164, -117.2164, -117.2164,
        -117.2164, -117.2164, -117.2164, -117.2164, -117.2164, -117.2164,
        -117.2164, -117.2164, -117.2164, -117.2164, -117.2164, -117.2164,
        -117.2164, -117.2164, -117.2164, -118.8222, -118.8222, -118.8222,
        -118.8222, -118.8222, -118.822